<a href="https://colab.research.google.com/github/harshyadav1508/GAN/blob/main/WGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#
%cd "/content/drive/MyDrive/Colab Notebooks/GAN/WGAN"
!pwd

/content/drive/MyDrive/Colab Notebooks/GAN/WGAN
/content/drive/MyDrive/Colab Notebooks/GAN/WGAN


In [30]:
import pandas as pd
from keras import backend
from keras.constraints import Constraint
from keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, BatchNormalization
from keras.initializers import RandomNormal
import numpy as np
from numpy import ones, mean, expand_dims, reshape
from numpy.random import randn, randint
from matplotlib import pyplot

# Clip model weights to a given hypercube

In [3]:
class ClipConstraint(Constraint):

  def __init__(self, clip_value):
    self.clip_value=clip_value

  def __call__(self,weights):
    return backend.clip(weights,-self.clip_value,self.clip_value)

  def get_config(self):
    return {'clip_value':self.clip_value}

# Calculate wasserstein loss

In [4]:
def wasserstein_loss(y_true, y_pred):
  return backend.mean(y_true*y_pred)

# Constant

In [5]:
init = RandomNormal(stddev=0.02)
const= ClipConstraint(0.01)
opt = RMSprop(lr=0.00005)
latent_dim=50

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


# Define the standalone critic model

In [6]:
def define_critic(in_shape=(28,28,1)):

  model=Sequential([
      Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init, kernel_constraint=const, input_shape=in_shape, activation=LeakyReLU(alpha=0.2)),
      BatchNormalization(momentum=0.8),
      Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init, kernel_constraint=const, activation=LeakyReLU(alpha=0.2)),
      BatchNormalization(momentum=0.8),
      Flatten(),
      Dense(1)
  ])

  model.compile(loss=wasserstein_loss, optimizer=opt)

  return model


# Define the standalone generator model

In [7]:
from keras.backend import tanh
def define_generator(latent_dim):

  n_nodes=7*7*128
  model=Sequential(
      [
          Dense(n_nodes, kernel_initializer=init,input_dim=latent_dim,activation=LeakyReLU(alpha=0.2)),
          Reshape((7,7,128)),
          Conv2DTranspose(128,(4,4),padding='same', strides=(2,2), kernel_initializer=init, activation=LeakyReLU(alpha=0.2)),
          BatchNormalization(momentum=0.8),
          Conv2DTranspose(128,(4,4),padding='same', strides=(2,2), kernel_initializer=init, activation=LeakyReLU(alpha=0.2)),
          BatchNormalization(momentum=0.8),
          Conv2D(1,(7,7),padding='same', kernel_initializer=init, activation='tanh')
      ]
  )

  model.summary()
  return model

In [8]:
critic = define_generator(100)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6272)              633472    
                                                                 
 reshape (Reshape)           (None, 7, 7, 128)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 14, 14, 128)      262272    
 nspose)                                                         
                                                                 
 batch_normalization (BatchN  (None, 14, 14, 128)      512       
 ormalization)                                                   
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 28, 28, 128)      262272    
 ranspose)                                                       
                                                        

# Combined generator and critic model,

In [45]:
def define_gan(generator, critic):


  critic.trainable=False

  

  model=Sequential([generator,critic])
      
  
  model.compile(loss=wasserstein_loss,optimizer=opt)
  return model

In [10]:
def load_real_samples():
  (trainX,trainy),(_,_)=load_data()
  select_ix=trainy==6
  X=trainX[select_ix]
  X=expand_dims(X,-1)
  X=X.astype('float')
  X=(X-127.5)/127.5

  return X


# Select real samples

In [23]:
def generate_real_samples(dataset,n_samples):
  ix=randint(0,dataset.shape[0],n_samples)
  X=dataset[ix]
  y=-np.ones((n_samples,1))

  return X,y


# generate points in latent space as input for the generator

In [34]:
def generate_latent_points(latent_dim,n_samples):
  x_input=randn(latent_dim*n_samples)
  x_input=x_input.reshape(n_samples,latent_dim)
  return x_input

# use the generator to generate n fake examples, with class labels

In [43]:
def generate_fake_samples(generator, latent_dim, n_samples):
  x_input=generate_latent_points(latent_dim,n_samples)
  X=generator.predict(x_input)
  y=np.ones((n_samples,1))

  return X,y


In [14]:
def summarize_performance(step, g_model, latent_dim, n_samples=100):

	X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
	X = (X + 1) / 2.0
	# plot images
	for i in range(10 * 10):
		pyplot.axis('off')
		pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
	# save plot to file
	filename1 = 'generated_plot_%04d.png' % (step+1)
	pyplot.savefig(filename1)
	pyplot.close()
	filename2 = 'model_%04d.h5' % (step+1)
	g_model.save(filename2)
	print('>Saved: %s and %s' % (filename1, filename2))

In [15]:
def plot_history(d1_hist, d2_hist, g_hist):
	# plot history
	pyplot.plot(d1_hist, label='crit_real')
	pyplot.plot(d2_hist, label='crit_fake')
	pyplot.plot(g_hist, label='gen')
	pyplot.legend()
	pyplot.savefig('plot_line_plot_loss.png')
	pyplot.close()

In [46]:
def train(g_model, c_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=64, n_critic=5):
  bat_per_epo = int(dataset.shape[0] / n_batch)
  n_steps = bat_per_epo * n_epochs
  half_batch = int(n_batch / 2)
  c1_hist, c2_hist, g_hist = list(), list(), list()

  for i in range(n_steps):
    c1_tmp, c2_tmp = list(), list()
    
    for _ in range(n_critic):
      X_real, y_real = generate_real_samples(dataset, half_batch)
      c_loss1 = c_model.train_on_batch(X_real, y_real)
      c1_tmp.append(c_loss1)

      X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
      c_loss2 = c_model.train_on_batch(X_fake, y_fake)
      c2_tmp.append(c_loss2)

    c1_hist.append(mean(c1_tmp))
    c2_hist.append(mean(c2_tmp))

    X_gan = generate_latent_points(latent_dim, n_batch)
    y_gan = -ones((n_batch, 1))

    g_loss = gan_model.train_on_batch(X_gan, y_gan)
    g_hist.append(g_loss)

		
    print('>%d, c1=%.3f, c2=%.3f g=%.3f' % (i+1, c1_hist[-1], c2_hist[-1], g_loss))

    if (i+1) % bat_per_epo == 0:
      summarize_performance(i, g_model, latent_dim)

    plot_history(c1_hist, c2_hist, g_hist)
		




In [47]:
latent_dim = 50
# create the critic
critic = define_critic()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, critic)
# load image data
dataset = load_real_samples()
print(dataset.shape)
# train model
train(generator, critic, gan_model, dataset, latent_dim)

Model: "sequential_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 6272)              319872    
                                                                 
 reshape_13 (Reshape)        (None, 7, 7, 128)         0         
                                                                 
 conv2d_transpose_26 (Conv2D  (None, 14, 14, 128)      262272    
 Transpose)                                                      
                                                                 
 batch_normalization_52 (Bat  (None, 14, 14, 128)      512       
 chNormalization)                                                
                                                                 
 conv2d_transpose_27 (Conv2D  (None, 28, 28, 128)      262272    
 Transpose)                                                      
                                                     

>Saved: generated_plot_0092.png and model_0092.h5
>93, c1=-89.293, c2=-1.910 g=-6.247
>94, c1=-89.964, c2=0.371 g=-6.228
>95, c1=-92.422, c2=0.616 g=-6.380
>96, c1=-92.361, c2=-0.239 g=-6.730
>97, c1=-93.723, c2=-1.814 g=-5.796
>98, c1=-94.880, c2=-3.836 g=-5.373
>99, c1=-96.011, c2=-2.904 g=-4.945
>100, c1=-96.235, c2=-4.561 g=-5.346
>101, c1=-97.635, c2=-5.625 g=-5.911
>102, c1=-98.026, c2=-4.821 g=-6.007
>103, c1=-98.713, c2=-5.662 g=-5.270
>104, c1=-100.124, c2=-7.101 g=-4.902
>105, c1=-99.890, c2=-7.368 g=-4.281
>106, c1=-101.845, c2=-8.019 g=-4.331
>107, c1=-102.778, c2=-8.394 g=-3.950
>108, c1=-102.462, c2=-7.443 g=-4.656
>109, c1=-102.435, c2=-9.244 g=-5.922
>110, c1=-103.431, c2=-8.958 g=-7.164
>111, c1=-105.056, c2=-8.623 g=-7.115
>112, c1=-105.285, c2=-9.734 g=-6.120
>113, c1=-107.147, c2=-7.569 g=-9.542
>114, c1=-106.256, c2=-8.625 g=-8.702
>115, c1=-108.435, c2=-6.874 g=-11.126
>116, c1=-109.376, c2=-8.076 g=-10.025
>117, c1=-109.212, c2=-7.261 g=-11.299
>118, c1=-110.904,

>Saved: generated_plot_0184.png and model_0184.h5
>185, c1=-157.167, c2=-29.037 g=-30.005
>186, c1=-161.794, c2=-41.407 g=-25.473
>187, c1=-161.843, c2=-28.911 g=-30.800
>188, c1=-163.754, c2=-46.918 g=-22.263
>189, c1=-161.946, c2=-27.463 g=-28.096
>190, c1=-165.099, c2=-42.291 g=-28.881
>191, c1=-165.427, c2=-32.463 g=-28.966
>192, c1=-165.427, c2=-38.181 g=-26.243
>193, c1=-167.955, c2=-37.926 g=-31.420
>194, c1=-167.261, c2=-41.354 g=-26.584
>195, c1=-172.166, c2=-45.018 g=-28.579
>196, c1=-169.875, c2=-36.549 g=-30.294
>197, c1=-170.337, c2=-30.640 g=-34.252
>198, c1=-170.451, c2=-33.990 g=-29.443
>199, c1=-173.782, c2=-45.609 g=-29.787
>200, c1=-174.677, c2=-41.323 g=-30.278
>201, c1=-173.022, c2=-42.525 g=-28.996
>202, c1=-174.755, c2=-37.167 g=-35.506
>203, c1=-173.712, c2=-46.415 g=-29.519
>204, c1=-177.509, c2=-42.083 g=-33.131
>205, c1=-177.353, c2=-37.910 g=-35.254
>206, c1=-178.167, c2=-54.744 g=-28.892
>207, c1=-181.110, c2=-56.338 g=-22.933
>208, c1=-184.024, c2=-55.643 

>Saved: generated_plot_0276.png and model_0276.h5
>277, c1=-240.991, c2=-95.617 g=-22.471
>278, c1=-245.119, c2=-111.137 g=-10.550
>279, c1=-248.304, c2=-112.230 g=-16.525
>280, c1=-245.459, c2=-87.795 g=-28.441
>281, c1=-245.073, c2=-96.858 g=-27.534
>282, c1=-243.428, c2=-95.571 g=-22.805
>283, c1=-250.869, c2=-120.784 g=-14.845
>284, c1=-251.063, c2=-105.951 g=-16.593
>285, c1=-248.203, c2=-83.961 g=-26.646
>286, c1=-250.822, c2=-109.635 g=-19.897
>287, c1=-250.624, c2=-100.356 g=-19.084
>288, c1=-252.129, c2=-92.164 g=-25.761
>289, c1=-251.613, c2=-95.675 g=-28.156
>290, c1=-251.559, c2=-81.707 g=-29.977
>291, c1=-248.463, c2=-96.971 g=-22.999
>292, c1=-256.048, c2=-116.264 g=-18.645
>293, c1=-256.250, c2=-104.980 g=-18.078
>294, c1=-257.279, c2=-113.113 g=-10.628
>295, c1=-259.204, c2=-102.488 g=-21.500
>296, c1=-259.638, c2=-123.267 g=-9.034
>297, c1=-262.851, c2=-121.545 g=-10.769
>298, c1=-264.972, c2=-121.007 g=-15.586
>299, c1=-262.284, c2=-105.193 g=-21.317
>300, c1=-259.410

>Saved: generated_plot_0368.png and model_0368.h5
>369, c1=-317.726, c2=-155.520 g=0.700
>370, c1=-327.212, c2=-161.016 g=-1.052
>371, c1=-319.256, c2=-187.017 g=13.898
>372, c1=-326.939, c2=-149.202 g=-12.278
>373, c1=-319.515, c2=-143.517 g=-8.143
>374, c1=-321.426, c2=-148.796 g=-4.909
>375, c1=-319.265, c2=-152.742 g=-10.334
>376, c1=-327.784, c2=-149.588 g=-8.198
>377, c1=-324.985, c2=-139.762 g=-10.009
>378, c1=-330.342, c2=-158.593 g=-1.858
>379, c1=-323.952, c2=-116.182 g=-38.619
>380, c1=-319.647, c2=-147.064 g=-5.035
>381, c1=-323.849, c2=-147.546 g=-9.319
>382, c1=-326.681, c2=-105.899 g=-19.373
>383, c1=-317.995, c2=-136.032 g=-1.442
>384, c1=-321.231, c2=-145.321 g=-3.749
>385, c1=-324.522, c2=-143.735 g=-10.071
>386, c1=-328.282, c2=-171.477 g=8.283
>387, c1=-329.107, c2=-163.453 g=7.303
>388, c1=-341.701, c2=-195.486 g=12.047
>389, c1=-343.061, c2=-198.109 g=18.255
>390, c1=-337.461, c2=-166.767 g=12.427
>391, c1=-353.120, c2=-180.692 g=-5.002
>392, c1=-336.964, c2=-142.

...

>Saved: generated_plot_0552.png and model_0552.h5
>553, c1=-418.072, c2=-153.018 g=10.339
>554, c1=-421.269, c2=-156.349 g=-2.575
>555, c1=-389.126, c2=-128.729 g=-10.201
>556, c1=-405.035, c2=-125.609 g=-43.394
>557, c1=-399.391, c2=-75.228 g=-68.931
>558, c1=-387.699, c2=-127.017 g=9.192
>559, c1=-420.846, c2=-165.835 g=18.887
>560, c1=-412.284, c2=-227.915 g=1.524
>561, c1=-417.778, c2=-195.079 g=20.165
>562, c1=-427.489, c2=-185.697 g=11.502
>563, c1=-428.072, c2=-219.871 g=21.428
>564, c1=-420.065, c2=-110.120 g=-66.489
>565, c1=-387.476, c2=-125.230 g=13.357
>566, c1=-424.710, c2=-133.408 g=-12.469
>567, c1=-397.254, c2=-198.057 g=4.523
>568, c1=-416.912, c2=-147.631 g=33.380
>569, c1=-423.749, c2=-197.101 g=4.350
>570, c1=-442.078, c2=-202.006 g=6.080
>571, c1=-403.621, c2=-139.817 g=-16.409
>572, c1=-402.691, c2=-38.909 g=-15.367
>573, c1=-398.503, c2=-152.018 g=12.376
>574, c1=-408.831, c2=-164.653 g=-13.420
>575, c1=-420.760, c2=-160.688 g=-13.769
>576, c1=-429.633, c2=-204.0

>Saved: generated_plot_0644.png and model_0644.h5
>645, c1=-356.906, c2=-88.913 g=-21.992
>646, c1=-377.834, c2=-74.812 g=-22.680
>647, c1=-383.146, c2=6.250 g=-108.800
>648, c1=-352.636, c2=-12.396 g=-71.680
>649, c1=-361.726, c2=16.989 g=-38.388
>650, c1=-371.746, c2=-80.637 g=-101.052
>651, c1=-359.131, c2=36.293 g=12.876
>652, c1=-329.245, c2=-72.358 g=-34.213
>653, c1=-361.316, c2=-106.630 g=-22.532
>654, c1=-374.440, c2=49.236 g=-17.363
>655, c1=-340.438, c2=1.534 g=-112.326
>656, c1=-342.657, c2=53.540 g=-1.137
>657, c1=-351.941, c2=-44.506 g=-140.806
>658, c1=-341.255, c2=52.701 g=-55.832
>659, c1=-334.839, c2=133.856 g=-101.331
>660, c1=-358.057, c2=37.784 g=-0.810
>661, c1=-350.603, c2=-102.872 g=77.359
>662, c1=-350.072, c2=-97.016 g=-7.239
>663, c1=-354.335, c2=-21.762 g=-3.538
>664, c1=-355.210, c2=-49.645 g=-77.252
>665, c1=-358.460, c2=71.141 g=-17.147
>666, c1=-361.461, c2=-5.072 g=-64.909
>667, c1=-315.677, c2=82.640 g=-131.541
>668, c1=-303.414, c2=92.317 g=-4.614
>66

>Saved: generated_plot_0736.png and model_0736.h5
>737, c1=-305.901, c2=289.953 g=-176.242
>738, c1=-346.745, c2=274.555 g=-321.983
>739, c1=-289.082, c2=342.693 g=-178.401
>740, c1=-342.684, c2=281.265 g=-294.430
>741, c1=-330.916, c2=292.022 g=-167.712
>742, c1=-316.240, c2=255.156 g=-282.498
>743, c1=-332.882, c2=330.190 g=-152.254
>744, c1=-325.562, c2=231.534 g=-168.764
>745, c1=-317.757, c2=278.700 g=-245.234
>746, c1=-304.083, c2=285.727 g=-167.357
>747, c1=-350.328, c2=269.665 g=-188.370
>748, c1=-292.522, c2=266.441 g=-291.093
>749, c1=-278.651, c2=366.237 g=-204.483
>750, c1=-325.847, c2=345.052 g=-253.928
>751, c1=-351.733, c2=308.796 g=-302.352
>752, c1=-324.228, c2=359.751 g=-233.147
>753, c1=-345.546, c2=321.809 g=-259.350
>754, c1=-334.633, c2=320.471 g=-326.032
>755, c1=-373.931, c2=318.432 g=-232.026
>756, c1=-344.643, c2=323.773 g=-359.263
>757, c1=-356.756, c2=370.539 g=-314.236
>758, c1=-366.193, c2=365.324 g=-375.552
>759, c1=-400.898, c2=360.260 g=-226.933
>760, c

>Saved: generated_plot_0828.png and model_0828.h5
>829, c1=-471.881, c2=454.724 g=-410.394
>830, c1=-472.067, c2=438.215 g=-416.050
>831, c1=-483.945, c2=445.060 g=-445.818
>832, c1=-488.523, c2=442.665 g=-378.987
>833, c1=-480.404, c2=434.870 g=-444.294
>834, c1=-495.293, c2=446.336 g=-374.019
>835, c1=-458.407, c2=444.714 g=-459.640
>836, c1=-476.188, c2=457.973 g=-416.085
>837, c1=-466.221, c2=439.629 g=-471.138
>838, c1=-478.125, c2=460.386 g=-451.941
>839, c1=-488.092, c2=464.227 g=-394.487
>840, c1=-473.001, c2=437.006 g=-437.973
>841, c1=-489.050, c2=434.606 g=-383.595
>842, c1=-475.372, c2=410.020 g=-454.382
>843, c1=-489.529, c2=454.138 g=-433.386
>844, c1=-474.428, c2=456.164 g=-412.155
>845, c1=-477.807, c2=444.807 g=-392.785
>846, c1=-482.332, c2=424.881 g=-367.088
>847, c1=-478.666, c2=404.118 g=-411.965
>848, c1=-472.878, c2=429.891 g=-472.956
>849, c1=-482.336, c2=450.620 g=-411.889
>850, c1=-484.691, c2=453.188 g=-466.652
>851, c1=-483.528, c2=448.876 g=-370.185
>852, c

>Saved: generated_plot_0920.png and model_0920.h5
